In [1]:
!pip uninstall -y numpy
!pip install "numpy<2.0" --no-cache-dir --force-reinstall

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 134.4 MB/s eta 0:00:00


In [1]:
import numpy as np
print(np.__version__)

1.26.4


In [2]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [2]:
!conda install -c pytorch faiss-gpu -y

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [3]:
import os
import pandas as pd
import numpy as np
import faiss
import re
import torch
import psycopg2
import io
from sentence_transformers import SentenceTransformer
from google.colab import drive
from google.colab import auth

In [5]:
import pickle

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
!curl ifconfig.me

curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
35.240.167.120

In [9]:
# Mount Google Drive
drive.mount('/content/drive')

# 🔹 Đường dẫn thư mục trên Google Drive
DATA_DIR = "/content/drive/MyDrive/amazon_data"
os.makedirs(DATA_DIR, exist_ok=True)

CSV_FILE = os.path.join(DATA_DIR, "amazon.csv")
EMBEDDING_FILE = os.path.join(DATA_DIR, "embeddings.npy")
FAISS_INDEX_FILE = os.path.join(DATA_DIR, "faiss_index.bin")

# 🔹 Thông tin kết nối PostgreSQL
DB_CONFIG = {
    "dbname": "your_database",
    "user": "your_user",
    "password": "your_password",
    "host": "localhost",
    "port": "5432"
}

def connect_db():
    """ Kết nối PostgreSQL trên Google Cloud SQL """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        conn.autocommit = True
        print("✅ Database conntected successfully.")
        return conn
    except psycopg2.Error as e:
        print(f"❌ Lỗi kết nối PostgreSQL: {e}")
        return None

# 🔹 Tạo bảng trong Cloud SQL
def create_tables(conn):
    """ Tạo bảng theo Star Schema """
    with conn.cursor() as cur:
        cur.execute("""
        DROP TABLE IF EXISTS fact_reviews, dim_products, dim_categories CASCADE;

        CREATE TABLE dim_categories (
            category_id SERIAL PRIMARY KEY,
            main_category TEXT NOT NULL,
            sub_category TEXT NOT NULL,
            UNIQUE(main_category, sub_category)
        );

        CREATE TABLE dim_products (
            product_id SERIAL PRIMARY KEY,
            name TEXT NOT NULL,
            category_id INT REFERENCES dim_categories(category_id),
            image TEXT,
            link TEXT,
            UNIQUE(name, category_id)
        );

        CREATE TABLE fact_reviews (
            review_id SERIAL PRIMARY KEY,
            product_id INT REFERENCES dim_products(product_id),
            rating NUMERIC(3,2),
            rating_count INT,
            actual_price NUMERIC(12,2),
            discount_price NUMERIC(12,2)
        );
        """)
        print("✅ Tables created successfully.")

def clean_data(df):
    """Làm sạch và chuẩn hóa dữ liệu từ CSV"""
    df = df.copy()

    # Chuẩn hóa tên cột
    df["main_category"] = df["main_category"].astype(str).str.strip().str.title()
    df["sub_category"] = df["sub_category"].astype(str).str.strip().str.title()
    df["name"] = df["name"].astype(str).str.strip()

    # Xóa ký tự ₹ và dấu phẩy trong giá tiền, chuyển thành số
    def safe_float_convert(value):
        try:
            return float(re.sub(r"[₹,]", "", str(value))) if value not in ["", "nan", None] else None
        except ValueError:
            return None

    df["actual_price"] = df["actual_price"].apply(safe_float_convert)
    df["discount_price"] = df["discount_price"].apply(safe_float_convert)

    # Làm sạch cột no_of_ratings
    def clean_ratings(value):
        if isinstance(value, str) and value.replace(",", "").isdigit():
            return int(value.replace(",", ""))
        return None  # Trả về None thay vì 0 để lọc bỏ khi dropna

    df["no_of_ratings"] = df["no_of_ratings"].apply(clean_ratings)

    # Làm sạch cột ratings
    def clean_ratings_value(value):
        try:
            return float(value)
        except (ValueError, TypeError):
            return None  # Trả về None thay vì 0 để lọc bỏ khi dropna

    df["ratings"] = df["ratings"].apply(clean_ratings_value)

    # 🔹 Loại bỏ hàng có NaN ở các cột quan trọng
    df = df.dropna(subset=["actual_price", "discount_price", "ratings", "no_of_ratings"])

    return df


def refactor_data(df):
    """ Refactor data without importing from PostgreSQL """

    # 🔹 Simulate category mapping
    category_map = {
        f"{row['main_category']}|{row['sub_category']}": idx
        for idx, row in enumerate(df[["main_category", "sub_category"]].drop_duplicates().to_dict(orient="records"), start=1)
    }

    df["category_id"] = df.apply(lambda x: category_map.get(f"{x['main_category']}|{x['sub_category']}"), axis=1)
    df = df.dropna(subset=["category_id"])

    # 🔹 Simulate product mapping
    product_map = {
        row["name"].lower(): idx
        for idx, row in enumerate(df[["name"]].drop_duplicates().to_dict(orient="records"), start=1)
    }

    df["product_id"] = df["name"].str.lower().map(product_map)
    df = df.dropna(subset=["product_id"])

    print("✅ Data refactored successfully.")
    return df

def import_data(conn, df):
    """ Nhập dữ liệu vào PostgreSQL """
    with conn.cursor() as cur:
        # 🔹 Chèn dữ liệu vào dim_categories
        cur.executemany("""
        INSERT INTO dim_categories (main_category, sub_category)
        VALUES (%s, %s) ON CONFLICT DO NOTHING;
        """, df[["main_category", "sub_category"]].drop_duplicates().values.tolist())

        # 🔹 Lấy category_id từ PostgreSQL
        cur.execute("SELECT category_id, main_category, sub_category FROM dim_categories;")
        category_map = {f"{row[1]}|{row[2]}": row[0] for row in cur.fetchall()}
        df["category_id"] = df.apply(lambda x: category_map.get(f"{x['main_category']}|{x['sub_category']}", None), axis=1)
        df = df.dropna(subset=["category_id"])

        # 🔹 Chèn dữ liệu vào dim_products
        cur.executemany("""
        INSERT INTO dim_products (name, category_id, image, link)
        VALUES (%s, %s, %s, %s) ON CONFLICT DO NOTHING;
        """, df[["name", "category_id", "image", "link"]].drop_duplicates().values.tolist())

        # 🔹 Lấy product_id từ PostgreSQL
        cur.execute("SELECT product_id, LOWER(name) FROM dim_products;")
        product_map = {row[1]: row[0] for row in cur.fetchall()}
        df["product_id"] = df["name"].str.lower().map(product_map)
        df = df.dropna(subset=["product_id"])

        # 🔹 Chèn dữ liệu vào fact_reviews
        cur.executemany("""
        INSERT INTO fact_reviews (product_id, rating, rating_count, actual_price, discount_price)
        VALUES (%s, %s, %s, %s, %s);
        """, df[["product_id", "ratings", "no_of_ratings", "actual_price", "discount_price"]].values.tolist())

        print("✅ Data imported successfully.")
        return df

def create_faiss_index(df):
    """Tạo FAISS index từ dữ liệu"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

    print("🔹 Đang tạo embeddings...")
    all_embeddings = model.encode(df["name"].str.lower().tolist(), device=device, show_progress_bar=True).astype('float32')
    np.save(EMBEDDING_FILE, all_embeddings)

    dimension = all_embeddings.shape[1]

    # FAISS setup
    index = faiss.IndexFlatL2(dimension)
    if torch.cuda.is_available():
        res = faiss.StandardGpuResources()
        index = faiss.index_cpu_to_gpu(res, 0, index)

    index.add(all_embeddings)
    index = faiss.index_gpu_to_cpu(index)
    faiss.write_index(index, FAISS_INDEX_FILE)
    print(f"✅ FAISS index đã lưu tại {FAISS_INDEX_FILE}")

    # 🔹 Load lại FAISS index để kiểm tra
    index = faiss.read_index(FAISS_INDEX_FILE)
    if torch.cuda.is_available():
        gpu_index = faiss.index_cpu_to_gpu(res, 0, index)
    else:
        gpu_index = index

    # 🔹 Truy vấn thử nghiệm
    query = "iphone 13"
    query_embedding = model.encode([query], device=device).astype('float32')
    gpu_index.nprobe = 10  # Tăng nprobe để cải thiện độ chính xác

    k = 10
    distances, indices = gpu_index.search(query_embedding, k)

    print(f"🔍 Kết quả tìm kiếm cho '{query}':")
    for i in range(k):
        idx = indices[0][i]
        if idx < len(df):  # Kiểm tra chỉ số hợp lệ
            print(f"{i+1}. {df['name'].iloc[idx]} (Khoảng cách: {distances[0][i]:.4f})")

    return FAISS_INDEX_FILE

def main():
    # conn = connect_db()
    # if not conn:
    #     return

    # create_tables(conn)

    df = pd.read_csv(CSV_FILE, encoding="utf-8")
    # 🔹 Làm sạch dữ liệu
    df_cleaned = clean_data(df)

    # # 🔹 Import data
    # df_final = import_data(conn, df_cleaned)

    # 🔹 Xử lý dữ liệu giống như khi import
    df_final = refactor_data(df_cleaned)

    # 🔹 Tạo FAISS index
    faiss_index_path = create_faiss_index(df_final)

    # conn.close()
    print("✅ Hoàn tất quá trình xử lý!")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Data refactored successfully.
🔹 Đang tạo embeddings...


Batches:   0%|          | 0/10467 [00:00<?, ?it/s]

✅ FAISS index đã lưu tại /content/drive/MyDrive/amazon_data/faiss_index.bin
🔍 Kết quả tìm kiếm cho 'iphone 13':
1. Apple iPhone 13 (256GB) - Midnight (Khoảng cách: 0.6427)
2. Apple iPhone 13 (128GB) - Midnight (Khoảng cách: 0.6601)
3. Apple iPhone 13 (256GB) - Blue (Khoảng cách: 0.7729)
4. Pinnaclz Speed 350IP Combo of USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7,... (Khoảng cách: 0.7774)
5. Apple iPhone 13 (128GB) - Blue (Khoảng cách: 0.7834)
6. Apple iPhone 13 (128GB) - Blue (Khoảng cách: 0.7834)
7. SupCares Edge to Edge Privacy Tempered Glass for iPhone 14, iPhone 13 and iPhone 13 Pro (6.1 Inch) with Easy Self Installa... (Khoảng cách: 0.7909)
8. Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini... (Khoảng cách: 0.7948)
9. Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini... (Khoảng cách: 0.79